<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/langchain_fulltext_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j openai langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.6 MB/s eta 0:00:00
 

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://3.238.222.252:7687",
    username="neo4j",
    password="bushels-harpoon-deduction"
)

In [9]:
import os

os.environ['OPENAI_API_KEY'] = "sk-"

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
    return_direct=True
)

In [10]:
chain.run("Who played in the Matrix?")



> Entering new  chain...
Generated Cypher:
MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie)
WHERE m.title = 'The Matrix'
RETURN a.name

> Finished chain.


[]

In [14]:
graph.query("""CREATE FULLTEXT INDEX entities IF NOT EXISTS
FOR (n:Movie|Person)
ON EACH [n.name, n.title]
""")

[]

In [16]:
graph.query("""
CALL db.index.fulltext.queryNodes("entities", "Matrix") YIELD node, score
RETURN node.title, score
LIMIT 3""")

[{'node.title': 'Matrix, The', 'score': 4.259097099304199},
 {'node.title': 'Matrix Revolutions, The', 'score': 3.7098255157470703},
 {'node.title': 'Matrix Reloaded, The', 'score': 3.7098255157470703}]

In [45]:
from __future__ import annotations
from langchain.chains.base import Chain

from typing import Any, Dict, List

from pydantic import Field


fulltext_search = """
CALL db.index.fulltext.queryNodes("entities", $query)
YIELD node, score
RETURN coalesce(node.title, node.name) AS option, labels(node)[0] AS type LIMIT 1
"""


class Neo4jFulltextGraphChain(Chain):
    """Chain for keyword question-answering against a graph."""

    graph: Neo4jGraph = Field(exclude=True)
    input_key: str = "query"  #: :meta private:
    output_key: str = "result"  #: :meta private:

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.
        :meta private:
        """
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        """Return the output keys.
        :meta private:
        """
        _output_keys = [self.output_key]
        return _output_keys

    def _call(self, inputs: Dict[str, str]) -> Dict[str, Any]:
        """Extract entities, look up info and answer question."""
        question = inputs[self.input_key]
        context = self.graph.query(
            fulltext_search, {'query': question})
        return {self.output_key: context}


In [46]:
fulltext = Neo4jFulltextGraphChain(graph=graph)
fulltext.run("Matrix")

[{'option': 'Matrix, The', 'type': 'Movie'}]

In [47]:
fulltext.run("Keanu")

[{'option': 'Keanu', 'type': 'Movie'}]

In [62]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
tools = [
    Tool(
        name="Movies",
        func=chain.run,
        description="""useful for when you need to retrieve information about movies, actors, directors, and similar.
        Input must be full question""",
    ),
    Tool(
        name="Entity search",
        func=fulltext.run,
        description="""useful for when you need find specific values of entities that appear in the question. Use exact values you retrieve from entity search as input to movies tool.
        Input must be a single entity.""",
    ),

]

In [63]:
graph_agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [64]:
graph_agent.run("Who played in the Matrix?")



> Entering new  chain...
Thought: I should use the Movies tool to retrieve the cast of the Matrix movie.
Action:
```
{
  "action": "Movies",
  "action_input": "cast of the Matrix"
}
```


> Entering new  chain...
Generated Cypher:
MATCH (m:Movie {title: 'The Matrix'})<-[:ACTED_IN]-(a:Actor)
RETURN a.name

> Finished chain.

Observation: []
Thought:It seems like the input to the Movies tool was not specific enough. Let me try again with a more specific input.
Action:
```
{
  "action": "Movies",
  "action_input": "cast of the Matrix (1999)"
}
```



> Entering new  chain...
Generated Cypher:
MATCH (m:Movie {title: 'The Matrix', year: 1999})-[:ACTED_IN]->(a:Actor)
RETURN a.name

> Finished chain.

Observation: []
Thought:

KeyboardInterrupt: ignored